<a href="https://colab.research.google.com/github/Abhishek1236/Machine-Learning-Web-Apps/blob/master/time_series_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import pandas_profiling as pp
import seaborn as sns
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from statsmodels.tsa.arima_model import ARIMA

In [4]:
from google.colab import files
uploaded = files.upload()

Saving mota.csv to mota.csv


In [0]:
train = pd.read_csv('mota.csv')

In [22]:
train.head()

,Unnamed: 0,Date,Mota_price
0,0,01-01-2019,67.247011
1,1,02-01-2019,67.018918
2,2,03-01-2019,67.047017
3,3,04-01-2019,67.022979
4,4,05-01-2019,67.039044


In [8]:
train.columns

Index(['Unnamed: 0', 'Date', 'Mota_price'], dtype='object')

In [0]:
train.drop(['Unnamed: 0'],axis =1 ,inplace = True)

In [24]:
train.isna().sum()

Date          0
Mota_price    0
dtype: int64

In [0]:
train = train.set_index('Date')

In [0]:

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return np.array(diff)


In [0]:

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

In [37]:
# seasonal difference
X = train.values
days_in_year = 90
differenced = difference(X, days_in_year)
# fit model
model = ARIMA(differenced, order=(7,0,1))
model_fit = model.fit(disp=0)
# print summary of fit model
print(model_fit.summary())

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                  275
Model:                     ARMA(7, 1)   Log Likelihood                 251.495
Method:                       css-mle   S.D. of innovations              0.093
Date:                Tue, 17 Mar 2020   AIC                           -482.989
Time:                        17:11:06   BIC                           -446.822
Sample:                             0   HQIC                          -468.474
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6469      0.062    -58.402      0.000      -3.769      -3.525
ar.L1.y        0.7082      0.039     18.275      0.000       0.632       0.784
ar.L2.y        0.2505      0.036      7.030      0.0

In [0]:
forecast = model_fit.forecast()[0]

In [41]:

from pandas import datetime
start_index = datetime('2020-01-20')
end_index = datetime('2020-01-26')
forecast = model_fit.predict(start=start_index, end=end_index)

TypeError: ignored

In [14]:
train.tail()

,Mota_price
Date,
27-12-2019,52.608735
28-12-2019,52.378117
29-12-2019,52.467574
30-12-2019,52.425681
31-12-2019,52.437536


In [0]:
mota= train.to_numpy()

In [16]:
# Building a stepwise model on train data 
stepwise_model = auto_arima(train, start_p=1, start_q=1,
                           max_p=4, max_q=4, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

Performing stepwise search to minimize aic
Fit ARIMA: (1, 1, 1)x(0, 1, 1, 12) (constant=True); AIC=-1034.882, BIC=-1015.564, Time=4.258 seconds
Fit ARIMA: (0, 1, 0)x(0, 1, 0, 12) (constant=True); AIC=-484.240, BIC=-476.512, Time=0.215 seconds
Fit ARIMA: (1, 1, 0)x(1, 1, 0, 12) (constant=True); AIC=-3107.133, BIC=-3091.678, Time=5.705 seconds
Near non-invertible roots for order (1, 1, 0)(1, 1, 0, 12); setting score to inf (at least one inverse root too close to the border of the unit circle: 1.000)
Fit ARIMA: (0, 1, 1)x(0, 1, 1, 12) (constant=True); AIC=-915.551, BIC=-900.097, Time=3.218 seconds
Near non-invertible roots for order (0, 1, 1)(0, 1, 1, 12); setting score to inf (at least one inverse root too close to the border of the unit circle: 0.999)
Fit ARIMA: (0, 1, 0)x(0, 1, 0, 12) (constant=False); AIC=-486.240, BIC=-482.376, Time=0.061 seconds
Fit ARIMA: (1, 1, 1)x(0, 1, 0, 12) (constant=True); AIC=-606.811, BIC=-591.357, Time=1.322 seconds
Near non-invertible roots for order (1, 

In [17]:
# Fitting on train
stepwise_model.fit(train)

ARIMA(maxiter=50, method='lbfgs', order=(1, 1, 1), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(1, 1, 2, 12),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)

In [0]:
import pickle
with open('model.pkl','wb') as file:
    pickle.dump(stepwise_model, file)

In [19]:
# Prediction & evaluation on validation
future_forecast = stepwise_model.predict(n_periods= 154)
tan_preds = future_forecast ; print(tan_preds)

[52.3785821  52.30372283 52.66962145 51.98879416 52.06103436 51.68790547
 51.46049837 51.96097135 51.74776666 52.0258558  51.96027912 51.98502791
 52.00831296 51.88728791 52.27717537 51.69456854 51.75697116 51.27364884
 50.91918403 51.51655548 51.28561494 51.37736588 51.33452656 51.34599851
 51.28761579 51.21148341 51.57712419 50.89706963 50.96857012 50.59327038
 50.36346395 50.86469584 50.65063584 50.92548079 50.85961999 50.88357212
 50.90509634 50.78409753 51.17303554 50.58844293 50.65036724 50.16796425
 49.81465472 50.41005924 50.17875013 50.27248198 50.22870988 50.23975071
 50.18189034 50.10451612 50.46987889 49.79056645 49.86132413 49.48390272
 49.2517428  49.75369861 49.53878731 49.81046426 49.74430831 49.76747044
 49.7872649  49.66627539 50.05427265 49.46773346 49.5291752  49.04764897
 48.69544477 49.28892069 49.05723596 49.1528761  49.1081801  49.11878484
 49.06141489 48.9828166  49.34789192 48.66928363 48.73930216 48.35980149
 48.12533696 48.62797921 48.41222332 48.68080406 48